Sometimes you don't want to use OpenRefine. Why not? I don't know, it's the *best* for cleaning up fuzzy matches. But yes, sure, sometimes maybe you don't.

In [61]:
%matplotlib inline
import pandas as pd

In [62]:
df = pd.read_csv("CD_Transactions_07-23-2017.CSV", index_col=False)

In [63]:
df.head()

,Result,Date,Transaction Type,Payment Type,Payment Detail,Amount,Last/Business Name,First Name,Address,City,...,--------,Report Type,Election Name,Election Type,Municipality,Office,Filer Type,Name,Report Year,Submitted
0,1,4/3/2017,Income,Check,12864,$350.00,Alaska Republican Party State Account,NaN,NaN,NaN,...,NaN,24 Hour Report,2017 - Anchorage Municipal Election,Anchorage Municipal,"Anchorage, Municipality of",School Board,Candidate,Kay E Schuster,2017,4/3/2017
1,2,4/3/2017,Income,Credit Card,NaN,$500.00,Coffey,Dan,NaN,NaN,...,NaN,24 Hour Report,2017 - Anchorage Municipal Election,Anchorage Municipal,"Anchorage, Municipality of",School Board,Candidate,Kay E Schuster,2017,4/3/2017
2,3,4/3/2017,Income,Check,3047,$300.00,ACS Employees PAC,NaN,NaN,NaN,...,NaN,24 Hour Report,2017 - Anchorage Municipal Election,Anchorage Municipal,"Anchorage, Municipality of",Assembly,Candidate,Pete Petersen,2017,4/3/2017
3,4,4/3/2017,Income,Credit Card,NaN,$500.00,Holmes,Patrick,NaN,NaN,...,NaN,24 Hour Report,2017 - Anchorage Municipal Election,Anchorage Municipal,"Anchorage, Municipality of",NaN,Group,Alaska Republican Party,2017,4/3/2017
4,5,4/3/2017,Income,Credit Card,NaN,$500.00,Gonzales,Mark L.,NaN,NaN,...,NaN,24 Hour Report,2017 - Anchorage Municipal Election,Anchorage Municipal,"Anchorage, Municipality of",NaN,Group,Alaska Republican Party,2017,4/3/2017


# What are all of our options for the "Alaska Sea Pilot PAC fund"?

In [65]:
df[df['Last/Business Name'] == 'Alaska Sea Pilot PAC fund'].shape

(3, 26)

In [67]:
df[df['Last/Business Name'] == 'ALASKA SEA PILOT PAC FUND'].shape

(6, 26)

In [69]:
df[df['Last/Business Name'] == 'Alaska Sea Pilot Pac Fund'].shape

(6, 26)

Maybe we can throw in a regex and catch some more?

In [70]:
df[df['Last/Business Name'].str.contains("Sea.*Pilot", na=False)]['Last/Business Name'].value_counts()

Alaska Sea Pilot PAC Fund                                     46
Alaska Sea Pilot PAC                                          17
Alaska Sea Pilots PAC Fund                                    13
Alaska Sea Pilot Pac Fund                                      6
Alaska Sea Pilots                                              5
Alaska Sea Pilot PAC fund                                      3
Alaska Sea Pilot Pac                                           3
Alaska Sea Pilot                                               3
Alaska Sea Pilots Pac Fund                                     2
Alaska Sea Pilot PAC Fund                                      2
Alaska Sea Pilot Fund                                          2
Alaska Sea Pilot  PAC fund                                     2
Alaska Sea Pilot PAC                                           1
Alaska Sea Pilots, 1621 Tongass Ave., Ketchikan, AK. 99901     1
 Ak Sea Pilot PAC                                              1
AlaskanSea Pilot PAC     

# Using fuzzywuzzy for finding fuzzy matches

Fuzzy matches are incomplete or inexact matches. The Python package [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy) has a few functions that can help you, although they're a little bit confusing! I'm going to take the examples from GitHub and annotate them a little, then we'll use them.

First, install fuzzywuzzy with

```bash
pip3 install fuzzywuzzy[speedup]
```

Then we'll get to importing it

In [ ]:
# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz

# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process

> **MAKE SURE YOU INSTALLED USING `pip3 install fuzzywuzzy[speedup]` OR ELSE IT WILL COMPLAIN HERE AND WILL ALSO BE SLOWER**

### `fuzz.ratio` compares the entire string, in order

Every single thing in the string is important here!

In [79]:
fuzz.ratio("this is a test", "this is a fun")

74

### `fuzz.partial_ratio` compares subsections of the string

Partial matches are fine! The exclamation mark at the end made `fuzz.ratio` not like the comparison last time, but this time it's OK.

In [84]:
fuzz.partial_ratio("this is a test", "test a is this")

57

### `fuzz.token_sort_ratio` ignores word order

`fuzz.token_sort_ratio` orders all of the words first, so "KENNEDY JOHN" and "JOHN KENNEDY" would be the same.

In [85]:
fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

100

In [86]:
fuzz.token_sort_ratio("this is a test", "is this a test")

100

In [88]:
fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

84

### `fuzz.token_set_ratio` ignores duplicate words

I don't know why you'd ever have "JOHN KENNEDY KENNEDY" but if you use `fuzz.token_set_ratio` then it would definitely match "JOHN KENNEDY".

In [91]:
fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

100

# Actually using fuzzywuzzy on our dataset, featuring `process.extract`

In [94]:
choices = ['fuzzy fuzzy was a bear', 'is this a test', 'THIS IS A TEST!!']
process.extract("this is a test", choices, scorer=fuzz.ratio)

[('THIS IS A TEST!!', 100),
 ('is this a test', 86),
 ('fuzzy fuzzy was a bear', 33)]

In [95]:
choices = ['fuzzy fuzzy was a bear', 'is this a test', 'THIS IS A TEST!!']
process.extract("this is a test", choices, scorer=fuzz.token_sort_ratio)

[('is this a test', 100),
 ('THIS IS A TEST!!', 100),
 ('fuzzy fuzzy was a bear', 28)]

Since we already imported, let's collect all of the business names into a list. We're going to search through the list to find names that are similar to **Alaska Sea Pilot PAC Fun**.

In [96]:
# If we grab a column and use .unique(), it gives us every business name with no repeats
choices = df['Last/Business Name'].unique()
choices[:15]

array(['Alaska Republican Party State Account', 'Coffey',
       'ACS Employees PAC', 'Holmes', 'Gonzales',
       'Anchorage Taxicab Permit Owners Association (ATPOA)', 'Abdullah',
       'Alimi', 'Barbosa', 'Bryant', 'Chamot', 'Farmer', 'Gautam',
       'Guevara', 'Lena'], dtype=object)

Now we'll use `process.extract` to find the top 15 matches

In [98]:
%%time
process.extract("Alaska Sea Pilot PAC Fund", choices, limit=30, scorer=fuzz.token_sort_ratio)

CPU times: user 634 ms, sys: 5.38 ms, total: 639 ms
Wall time: 642 ms


[('Alaska Sea Pilot PAC Fund', 100),
 ('Alaska Sea Pilot  PAC fund', 100),
 ('ALASKA SEA PILOT PAC FUND', 100),
 ('Alaska Sea Pilot PAC Fund ', 100),
 ('Alaska SEA Pilot Pac Fund', 100),
 ('Alaska SEA Pilot PAC Fund', 100),
 ('Alaska Sea Pilot Pac Fund', 100),
 ('Alaska Sea Pilot PAC fund', 100),
 ('Alaska Sea Pilots PAC Fund', 98),
 ('Alaska Sea Pilots Pac Fund', 98),
 ('Alaska Sea Pilot Fund', 91),
 ('AK Sea Pilot Pac Fund', 91),
 ('ALASKA SEA PILOT FUND', 91),
 ('AK Sea Pilot PAC Fund', 91),
 ('Alaska Sea Pilot Pac', 89),
 ('Alaska Sea Pilot PAC', 89),
 ('Alaska Sea Pilot Pac ', 89),
 ('Alaska Sea Pilot PAC ', 89),
 ('ALASK SEA PILOT PAC', 86),
 ('Alaska Sea Pilot', 78),
 ('AK Sea Pilot PAC', 78),
 (' Ak Sea Pilot PAC', 78),
 ('Alaska Sea Pilot ', 78),
 ('Alaska Sea Pilots, LLC.', 78),
 ('Alaska Sea Pilots', 76),
 ('AlaskanSea Pilot PAC', 76),
 ('Alaska Marine Pilot PAC', 75),
 ("Alaska Sea Pilots Ass'n", 75),
 ('Alaska SeaPilot PAC', 73),
 ('Alaska Senate Majority Fund', 73)]

Wow, those look pretty nice! Maybe instead we should just find the ones that are above a certain score? You can also specify a `scorer` if you want to get particular.

In [99]:
# Get 100 options so we're sure to have some non-matches
possibilities = process.extract("Alaska Sea Pilot PAC Fund", choices, limit=100, scorer=fuzz.token_sort_ratio)

In [100]:
# And let's see everyting with a score above 73
[possible for possible in possibilities if possible[1] > 73]

[('Alaska Sea Pilot PAC Fund', 100),
 ('Alaska Sea Pilot  PAC fund', 100),
 ('ALASKA SEA PILOT PAC FUND', 100),
 ('Alaska Sea Pilot PAC Fund ', 100),
 ('Alaska SEA Pilot Pac Fund', 100),
 ('Alaska SEA Pilot PAC Fund', 100),
 ('Alaska Sea Pilot Pac Fund', 100),
 ('Alaska Sea Pilot PAC fund', 100),
 ('Alaska Sea Pilots PAC Fund', 98),
 ('Alaska Sea Pilots Pac Fund', 98),
 ('Alaska Sea Pilot Fund', 91),
 ('AK Sea Pilot Pac Fund', 91),
 ('ALASKA SEA PILOT FUND', 91),
 ('AK Sea Pilot PAC Fund', 91),
 ('Alaska Sea Pilot Pac', 89),
 ('Alaska Sea Pilot PAC', 89),
 ('Alaska Sea Pilot Pac ', 89),
 ('Alaska Sea Pilot PAC ', 89),
 ('ALASK SEA PILOT PAC', 86),
 ('Alaska Sea Pilot', 78),
 ('AK Sea Pilot PAC', 78),
 (' Ak Sea Pilot PAC', 78),
 ('Alaska Sea Pilot ', 78),
 ('Alaska Sea Pilots, LLC.', 78),
 ('Alaska Sea Pilots', 76),
 ('AlaskanSea Pilot PAC', 76),
 ('Alaska Marine Pilot PAC', 75),
 ("Alaska Sea Pilots Ass'n", 75)]

**Huh, pretty neat.**

## Filtering directly with fuzzywuzzy

We can also use this directly with our dataframe, if we'd like to use `fuzzywuzzy` to filter instead of giving us a list. It's going to be a lot slower, but that's life, I guess.

In [101]:
def get_ratio(row):
    name = row['Last/Business Name']
    return fuzz.token_sort_ratio(name, "Alaska Sea Pilot PAC Fund")

df[df.apply(get_ratio, axis=1) > 70]

,Result,Date,Transaction Type,Payment Type,Payment Detail,Amount,Last/Business Name,First Name,Address,City,...,--------,Report Type,Election Name,Election Type,Municipality,Office,Filer Type,Name,Report Year,Submitted
50505,50506,12/3/2015,Income,Check,1278,$500.00,Alaska Sea Pilot PAC Fund,NaN,"1621 Tongass Ave, Ste 300",Ketchikan,...,NaN,Year Start Report,2016 - State Primary Election,State Primary,NaN,House,Candidate,Jonathan Kreiss-Tomkins,2016,6/13/2016
51557,51558,11/23/2015,Income,Check,1279,$500.00,Alaska Sea Pilot Pac,NaN,1621 Tongass Ave. Ste. 300,Ketchikan,...,NaN,Year Start Report,2016 - State Primary Election,State Primary,NaN,Senate,Candidate,Bert K. Stedman,2016,6/23/2016
61585,61586,11/21/2015,Income,Check,1090,$500.00,Alaska Sea Pilots PAC Fund,NaN,1621 Tongass Avenue Ste. 300,Ketchikan,...,NaN,Year Start Report,2016 - State Primary Election,State Primary,NaN,House,Candidate,Sam S Kito III,2016,8/9/2016
62351,62352,11/16/2015,Income,Check,1084,$500.00,Alaska Sea Pilot PAC,NaN,1621 Tongass Ave,Ketchikan,...,NaN,Year Start Report,2016 - State Primary Election,State Primary,NaN,House,Candidate,Neal Winston Foster,2016,8/28/2016
74155,74156,11/16/2015,Income,Check,1087,$500.00,Alaska Sea Pilot PAC Fund,NaN,1621 Tongass Ave,Ketchikan,...,NaN,Year Start Report,2016 - State Primary Election,State Primary,NaN,House,Candidate,Lynn Gattis,2016,10/9/2016
75904,75905,9/13/2016,Income,Check,1289,"$1,000.00",Alaska Sea Pilot PAC Fund,Alaska Sea Pilot PAC Fund,1621 Tongass Avenue,Ketchikan,...,NaN,Thirty Day Report,2016 - State General Election,State General,NaN,House,Candidate,"Andrew ""Andy"" Josephson",2016,10/9/2016
77090,77091,9/13/2016,Income,Check,1285,$500.00,Alaska Sea Pilot,NaN,"1621 Tongass Avenue, Suite 300",Ketchikan,...,NaN,Thirty Day Report,2016 - State General Election,State General,NaN,House,Candidate,Cathy L. Tilton,2016,10/10/2016
77110,77111,10/24/2016,Income,Check,1297,"$1,000.00",Alaska Sea Pilot PAC Fund,NaN,1621 Tongass Ave.,Ketchikan,...,NaN,Seven Day Report,2016 - State General Election,State General,NaN,House,Candidate,Dave Talerico,2016,10/31/2016
77259,77260,9/15/2016,Income,Check,1288,$500.00,ALASK SEA PILOT PAC,NaN,1621 TONGASS AVE,KETCHIKAN,...,NaN,Thirty Day Report,2016 - State General Election,State General,NaN,House,Candidate,Gary Allan Knopp,2016,10/10/2016
77473,77474,9/14/2016,Income,Check,1284,$500.00,Alaska Sea Pilot Fund,n/a,1621 Tongass Avenue,Ketchikan,...,NaN,Thirty Day Report,2016 - State General Election,State General,NaN,House,Candidate,cathy munoz,2016,10/10/2016


You could also do it using a lambda if you wanted

In [ ]:
df[df.apply(lambda row: fuzz.token_sort_ratio(row['Last/Business Name'], "Alaska Sea Pilot PAC Fund"), axis=1) > 70]

## We could technically clean it like below, but... it seems risky. Because it *is* risky!

```python
df.loc[df.apply(get_ratio, axis=1) > 75, "Last/Business Name"] = "Alaska Sea Pilot PAC Fund"
```